This EDA aims at:
1. understanding the data - identifying data types, identifying missing data, checking for outliers.
2. Cleaning up data, filling in missing data, changing feature data types and handling outliers

In [629]:
#importing libraries
import pandas as pd
import numpy as np 
import seaborn as sns
import missingno as msg
from matplotlib import pyplot as plt

In [630]:
# data path
data = r"carrier_kpi_messy_15000_FTL_LTL.csv"

In [631]:
# loading data to dataframe
df = pd.read_csv(data)

In [632]:
# looking at first 5 rows

df.head()

,carrier_name,carrier_number,shipment_number,shipment_date,delivery_date,delivery_reliability,feedback_reliability,non_conformance_count,shipment_weight_kg,cost_usd,transport_mode,status,region,remarks,data_entry_source
0,YRC Worldwide,930900,SHP782505,2024-12-13,2024-12-28,0,0,2,892,886.91,Air,Delivered,Middle East,Left at front desk -- see case #8054,EDI
1,DBSchenker,337136,SHP524303,2024-05-27,2024-06-04,1,0,0,244,401.23,Air,Delivered,Asia-Pacific,Damaged packaging,Manual
2,DB Schenker,291262,SHP472629,2023-07-17,2023-08-14,1,1,0,27529,22059.45,Rail,In Transit,Africa,Handled with care,Manual
3,COSCO,CR-704-C,SHP798739,2024-09-20,2024-09-30,0,0,2,3506,2772.15,Road,Delivered,Latin America,Customer not available,Email
4,Kuehne + Nagel,2663913,SHP443799,2025-08-12,2025-08-14,0,0,1,91682,$73373.86,Sea,Delivered,Africa,Received by warehouse,Manual


In [633]:
# checking df info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   carrier_name           15000 non-null  object
 1   carrier_number         14970 non-null  object
 2   shipment_number        15000 non-null  object
 3   shipment_date          14839 non-null  object
 4   delivery_date          14703 non-null  object
 5   delivery_reliability   15000 non-null  object
 6   feedback_reliability   14491 non-null  object
 7   non_conformance_count  15000 non-null  object
 8   shipment_weight_kg     14846 non-null  object
 9   cost_usd               14679 non-null  object
 10  transport_mode         15000 non-null  object
 11  status                 15000 non-null  object
 12  region                 15000 non-null  object
 13  remarks                13834 non-null  object
 14  data_entry_source      15000 non-null  object
dtypes: object(15)
memor

1. The delivery date will be used because the KPIs are only relevant after the goods are delivered. This is an object and must be converted into date format.
2. Delivery reliability is a boolean. Must be converted to INT.
3. Feedback reliability is a boolean. Must be converted to INT.
4. Non conformance count is a boolean. Must be converted to INT.
5. Shimpent weight kg is a float.
6. cost usd is a float

In [634]:
clean_df = df.copy()

1. Converting delivery date to date format

In [635]:
clean_df['delivery_date'].value_counts()

delivery_date
2023-07-30    26
2025-07-30    25
2024-10-20    24
2024-02-08    24
2023-02-07    24
              ..
2026-05-16     1
2028-06-23     1
2029-06-09     1
04/28/2024     1
2027-12-26     1
Name: count, Length: 1668, dtype: int64

#### It looks like the delivery date has multiple format types. 
#### I will convert them into a consistent format to enable converting them to date format

In [636]:
from datetime import datetime

def date_reformatter(date_value):

    """This function standardises inconsistent date formats"""
    if not date_value or not isinstance(date_value, str):
        return None  # handle nulls or non-strings

    # unify separators
    date_value = date_value.replace("/", "-")

    # try multiple formats
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%m-%d-%Y"):
        try:
            d = datetime.strptime(date_value, fmt)
            return d.strftime("%Y-%m-%d")  # normalized output
        except ValueError:
            continue  # try next format

    # fallback if none worked
    return date_value

In [637]:
#standardizing the date formats
clean_df["delivery_date"] = clean_df["delivery_date"].apply(date_reformatter)

In [638]:
# checking the data type
clean_df['delivery_date'].dtype

dtype('O')

In [639]:
# converting to date type
clean_df["delivery_date"] = pd.to_datetime(clean_df["delivery_date"])

In [640]:
# Spot Check
clean_df['delivery_date'].dtype

dtype('<M8[ns]')

2. Converting Delivery Reliability to boolean

In [641]:
df["delivery_reliability"].unique()

array(['0', '1', 'Yes', 'True', 'No', 'False'], dtype=object)

There are different types of unique values for "delivery_reliability". I simply need them to be a boolean. 
##### I will convert yes, true to 1 and no, false, nan to 0

In [642]:
df["delivery_reliability"].value_counts()

delivery_reliability
1        10789
0         2969
Yes        676
True       329
No         154
False       83
Name: count, dtype: int64

In [643]:
def boolean_standardiser(value):
    """Standardises various boolean-like inputs into either 1 or 0."""
    if value is None:
        return 0  # handle missing values

    # Convert to string and lowercase once
    val_str = str(value).strip().lower()

    if val_str in ("yes", "true", "1", "y", "t"):
        return 1
    else:
        return 0


In [644]:
#applying boolean standardiser to the delivery reliability column
df["delivery_reliability"] = df["delivery_reliability"].apply(boolean_standardiser)

In [645]:
# spot checks
df["delivery_reliability"].dtype

dtype('int64')

In [646]:
# spot checks
df["delivery_reliability"].value_counts()

delivery_reliability
1    11794
0     3206
Name: count, dtype: int64

3. Feedback reliability is a boolean. Must be converted to INT.

In [647]:
df["feedback_reliability"] = df["feedback_reliability"].apply(boolean_standardiser)

In [648]:
# spot checks
df["feedback_reliability"].dtype

dtype('int64')

In [649]:
df["feedback_reliability"].value_counts()

feedback_reliability
0    10085
1     4915
Name: count, dtype: int64

4. Non conformance count is a boolean. Must be converted to INT.

In [650]:
df['non_conformance_count'].value_counts()

non_conformance_count
0      12544
No       664
1        610
3        551
2        543
Yes       88
Name: count, dtype: int64

In [651]:
df["non_conformance_count"] = df['non_conformance_count'].apply(boolean_standardiser)

In [652]:
df['non_conformance_count'].dtype

dtype('int64')

In [653]:
df['non_conformance_count'].value_counts()

non_conformance_count
0    14302
1      698
Name: count, dtype: int64

In [654]:
df["shipment_weight_kg"]

0          892
1          244
2        27529
3         3506
4        91682
         ...  
14995    28163
14996    24840
14997    13067
14998    68419
14999    34988
Name: shipment_weight_kg, Length: 15000, dtype: object

5. Shipment weight kg is a float.

In [655]:
def weight_formatter(value):
    if value is None:
        return 0
    
    val_str = str(value).strip().lower()

    if "g" in val_str and "kg" not in val_str:
        formatted_val = float(val_str.replace("g","").strip())/1000
    else:
        formatted_val = float(val_str.replace("kg","").strip())  # handles kg and numbers

    return formatted_val
    


In [656]:
clean_df["shipment_weight_kg_f"] = clean_df["shipment_weight_kg"].apply(weight_formatter)

In [657]:
clean_df["shipment_weight_kg"].isna().sum()

np.int64(154)

In [658]:
clean_df.head()

,carrier_name,carrier_number,shipment_number,shipment_date,delivery_date,delivery_reliability,feedback_reliability,non_conformance_count,shipment_weight_kg,cost_usd,transport_mode,status,region,remarks,data_entry_source,shipment_weight_kg_f
0,YRC Worldwide,930900,SHP782505,2024-12-13,2024-12-28,0,0,2,892,886.91,Air,Delivered,Middle East,Left at front desk -- see case #8054,EDI,892.0
1,DBSchenker,337136,SHP524303,2024-05-27,2024-06-04,1,0,0,244,401.23,Air,Delivered,Asia-Pacific,Damaged packaging,Manual,244.0
2,DB Schenker,291262,SHP472629,2023-07-17,2023-08-14,1,1,0,27529,22059.45,Rail,In Transit,Africa,Handled with care,Manual,27529.0
3,COSCO,CR-704-C,SHP798739,2024-09-20,2024-09-30,0,0,2,3506,2772.15,Road,Delivered,Latin America,Customer not available,Email,3506.0
4,Kuehne + Nagel,2663913,SHP443799,2025-08-12,2025-08-14,0,0,1,91682,$73373.86,Sea,Delivered,Africa,Received by warehouse,Manual,91682.0


In [659]:
clean_df.groupby("transport_mode").agg({"shipment_weight_kg_f": "mean"})

,shipment_weight_kg_f
transport_mode,
AIR,497.385827
Air,526.286928
RAIL,23752.588785
ROAD,6178.352459
Rail,23940.269863
Road,6752.948953
SEA,58551.208696
Sea,60117.621499
air,548.125000


The above example also shows that transport mode column must be standardised


6. cost usd is a float

In [662]:
def cost_standardiser(cost):
    if cost is None:
        return None
    
    cost_str = str(cost).strip().upper()

   
    formatted_cost = cost_str.replace("$","").replace("USD","").replace("DOLLARS","").strip()
        
    formatted_cost = float(formatted_cost)

    return formatted_cost

        

In [663]:
clean_df['cost_usd'] = clean_df['cost_usd'].apply(cost_standardiser)

In [664]:
clean_df['cost_usd'].mean()

np.float64(18451.28232440902)